# Criar Corpus Específico do CohQuAD Inc en

Geração do corpus específico para o conjunto de dados.


Gera as POS-Tagging dos documentos perturbados do conjunto de dados.
- Utiliza dados textos sobre o assunto do conjunto de dados.
- Gera o arquivo `corpus_especifico.zip`.
 


# 1 Preparação do ambiente

Preparação do ambiente para execução do script.

## 1.1 Tempo inicial de processamento

In [1]:
# Import das bibliotecas.
import time
import datetime

# Marca o tempo de início do processamento
inicio_processamento = time.time()

## 1.2 Funções e classes auxiliares

Verifica se existe o diretório cohebert no diretório corrente.   


In [2]:
# Import das bibliotecas.
import os # Biblioteca para manipular arquivos

# ============================  
def verificaDiretorioCoheBERT():
    """
      Verifica se existe o diretório cohebert no diretório corrente.    
    """
    
    # Verifica se o diretório existe
    if not os.path.exists(DIRETORIO_COHEBERT):  
        # Cria o diretório
        os.makedirs(DIRETORIO_COHEBERT)
        logging.info("Diretório Cohebert criado: {}".format(DIRETORIO_COHEBERT))
    
    return DIRETORIO_COHEBERT

Realiza o download e um arquivo

In [3]:
# Import das bibliotecas.
import requests # Biblioteca de download
from tqdm.notebook import tqdm as tqdm_notebook # Biblioteca para barra de progresso
import os # Biblioteca para manipular arquivos

def downloadArquivo(url_arquivo, nome_arquivo_destino):
    """
      Realiza o download de um arquivo de uma url em salva em nome_arquivo_destino.
    
      Parâmetros:
        `url_arquivo` - URL do arquivo a ser feito download.      
        `nome_arquivo_destino` - Nome do arquivo a ser salvo.      
    """
    
    # Verifica se existe o diretório base
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()
    
    # Realiza o download de um arquivo em uma url
    data = requests.get(url_arquivo, stream=True)
    
    # Verifica se o arquivo existe
    if data.status_code != 200:
        logging.info("Exceção ao tentar realizar download {}. Response {}.".format(url_arquivo, data.status_code))
        data.raise_for_status()
        return

    # Recupera o nome do arquivo a ser realizado o download    
    nome_arquivo = nome_arquivo_destino.split("/")[-1]  

    # Define o nome e caminho do arquivo temporário    
    nome_arquivo_temporario = DIRETORIO_COHEBERT + "/" + nome_arquivo + "_part"
    
    logging.info("Download do arquivo: {}.".format(nome_arquivo_destino))
    
    # Baixa o arquivo
    with open(nome_arquivo_temporario, "wb") as arquivo_binario:        
        tamanho_conteudo = data.headers.get("Content-Length")        
        total = int(tamanho_conteudo) if tamanho_conteudo is not None else None
        # Barra de progresso de download
        progresso_bar = tqdm_notebook(unit="B", total=total, unit_scale=True)                
        # Atualiza a barra de progresso
        for chunk in data.iter_content(chunk_size=1024):        
            if chunk:                
                progresso_bar.update(len(chunk))
                arquivo_binario.write(chunk)
    
    # Renomeia o arquivo temporário para o arquivo definitivo
    os.rename(nome_arquivo_temporario, nome_arquivo_destino)
    
    # Fecha a barra de progresso.
    progresso_bar.close()

Remove tags de um documento

In [4]:
def remove_tags(documento):
    """
      Remove tags de um documento
    """
    
    import re

    documento_limpo = re.compile("<.*?>")
    return re.sub(documento_limpo, "", documento)

Funções auxiliares de arquivos

In [5]:
def carregar(nome_arquivo, encoding="Windows-1252"):
    """
      Carrega um arquivo texto e retorna as linhas como um único parágrafo(texto).
    
      Parâmetros:
        `nome_arquivo` - Nome do arquivo a ser carregado.  
    """

    # Abre o arquivo
    arquivo = open(nome_arquivo, "r", encoding= encoding)
    
    paragrafo = ""
    for linha in arquivo:
        linha = linha.splitlines()
        linha = " ".join(linha)
        # Remove as tags existentes no final das linhas
        linha = remove_tags(linha)
        if linha != "":
          paragrafo = paragrafo + linha.strip() + " "
    
    # Fecha o arquivo
    arquivo.close()

    # Remove os espaços em branco antes e depois do parágrafo
    return paragrafo.strip()

In [6]:
def carregarLista(nome_arquivo, encoding="Windows-1252"):
    """
      Carrega um arquivo texto e retorna as linhas como uma lista de sentenças(texto).
    
      Parâmetros:
        `nome_arquivo` - Nome do arquivo a ser carregado.   
        `encoding` - Codificação dos caracteres do arquivo.
    """

    # Abre o arquivo
    arquivo = open(nome_arquivo, "r", encoding= encoding)
    
    sentencas = []
    for linha in arquivo:        
        linha = linha.splitlines()
        linha = " ".join(linha)
        linha = remove_tags(linha)
        if linha != "":
          sentencas.append(linha.strip())
    
    # Fecha o arquivo
    arquivo.close()

    return sentencas 

In [7]:
def salvar(nome_arquivo,texto):                       
    """
      Salva um texto em arquivo.
     
      Parâmetros:
        `nome_arquivo` - Nome do arquivo a ser salvo.
        `texto` - Texto a ser salvo.     
    """

    arquivo = open(nome_arquivo, "w")
    arquivo.write(str(texto))
    arquivo.close()

Função auxiliar para formatar o tempo como `hh: mm: ss`

In [8]:
# Import das bibliotecas.
import time
import datetime

def formataTempo(tempo):
    """
      Pega a tempo em segundos e retorna uma string hh:mm:ss
    """
    # Arredonda para o segundo mais próximo.
    tempo_arredondado = int(round((tempo)))
    
    # Formata como hh:mm:ss
    return str(datetime.timedelta(seconds=tempo_arredondado))    

Classe(ModelArguments) de definição dos parâmetros do modelo

In [9]:
# Import das bibliotecas.
from dataclasses import dataclass, field
from typing import Dict, Optional
from typing import List

@dataclass
class ModeloArgumentosMedida:
    max_seq_len: Optional[int] = field(
        default=None,
        metadata={"help": "max seq len"},
    )    
    pretrained_model_name_or_path: str = field(
        default="neuralmind/bert-base-portuguese-cased",
        metadata={"help": "nome do modelo pré-treinado do BERT."},
    )
    modelo_spacy: str = field(
        default="pt_core_news_lg",
        metadata={"help": "nome do modelo do spaCy."},
    )
    versao_modelo_spacy: str = field(
        default="-3.2.0",
        metadata={"help": "versão do nome do modelo no spaCy."},
    )
    sentenciar_documento: bool = field(
        default=True,
        metadata={"help": "Dividir o documento em sentenças(frases)."},
    )
    do_lower_case: bool = field(
        default=False,
        metadata={"help": "define se o texto do modelo deve ser todo em minúsculo."},
    )    
    output_attentions: bool = field(
        default=False,
        metadata={"help": "habilita se o modelo retorna os pesos de atenção."},
    )
    output_hidden_states: bool = field(
        default=False,
        metadata={"help": "habilita gerar as camadas ocultas do modelo."},
    )
    usar_mcl_ajustado : bool = field(
        default=False,
        metadata={"help": "habilita o carragamento de mcl ajustado."},
    )
    documentos_perturbados: int = field(
        default="1",
        metadata={"help": "Quantidade de documentos a serem perturbados a partir do original."},
    )
    top_k_predicao: int = field(
        default="100",
        metadata={"help": "Quantidade de palavras a serem recuperadas mais próximas da máscara."},
    )    

Biblioteca de limpeza de tela


In [10]:
# Import das bibliotecas.
from IPython.display import clear_output

## 1.3 Tratamento de logs

In [11]:
# Import das bibliotecas.
import logging # Biblioteca de logging

# Formatando a mensagem de logging
logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s")

logger = logging.getLogger()
logger.setLevel(logging.INFO)

## 1.4 Identificando o ambiente Colab

In [12]:
# Import das bibliotecas.
import sys # Biblioteca para acessar módulos do sistema

# Se estiver executando no Google Colaboratory
# Retorna true ou false se estiver no Google Colaboratory
IN_COLAB = "google.colab" in sys.modules

## 1.5 Colaboratory

Usando Colab GPU para Treinamento


Uma GPU pode ser adicionada acessando o menu e selecionando:

`Edit -> Notebook Settings -> Hardware accelerator -> (GPU)`

Em seguida, execute a célula a seguir para confirmar que a GPU foi detectada.

In [13]:
# Import das bibliotecas.
import tensorflow as tf

# Recupera o nome do dispositido da GPU.
device_name = tf.test.gpu_device_name()

# O nome do dispositivo deve ser parecido com o seguinte:
if device_name == "/device:GPU:0":
    logging.info("Encontrei GPU em: {}".format(device_name))
else:
    logging.info("Dispositivo GPU não encontrado")
    #raise SystemError("Dispositivo GPU não encontrado")

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:root:Dispositivo GPU não encontrado


Nome da GPU

Para que a torch use a GPU, precisamos identificar e especificar a GPU como o dispositivo. Posteriormente, em nosso ciclo de treinamento, carregaremos dados no dispositivo.

Vale a pena observar qual GPU você recebeu. A GPU Tesla P100 é muito mais rápido que as outras GPUs, abaixo uma lista ordenada:
- 1o Tesla P100
- 2o Tesla T4
- 3o Tesla P4 (Não tem memória para execução 4 x 8, somente 2 x 4)
- 4o Tesla K80 (Não tem memória para execução 4 x 8, somente 2 x 4)

In [14]:
# Import das bibliotecas.
import torch

def getDeviceGPU():
    """
      Retorna um dispositivo de GPU se disponível ou CPU.
    
      Retorno:
        `device` - Um device de GPU ou CPU.       
    """
        
    # Se existe GPU disponível.
    if torch.cuda.is_available():
        
        # Diz ao PyTorch para usar GPU.    
        device = torch.device("cuda")
        
        logging.info("Existem {} GPU(s) disponíveis.".format(torch.cuda.device_count()))
        logging.info("Iremos usar a GPU: {}.".format(torch.cuda.get_device_name(0)))

    # Se não.
    else:        
        logging.info("Sem GPU disponível, usando CPU.")
        device = torch.device("cpu")
        
    return device

In [15]:
# Recupera o device com GPU ou CPU
device = getDeviceGPU()

INFO:root:Sem GPU disponível, usando CPU.


Memória

Memória disponível no ambiente

In [16]:
# Importando as bibliotecas.
from psutil import virtual_memory

ram_gb = virtual_memory().total / 1e9
logging.info("Seu ambiente de execução tem {: .1f} gigabytes de RAM disponível\n".format(ram_gb))

if ram_gb < 20:
  logging.info("Para habilitar um tempo de execução de RAM alta, selecione menu o ambiente de execução> \"Alterar tipo de tempo de execução\"")
  logging.info("e selecione High-RAM. Então, execute novamente está célula")
else:
  logging.info("Você está usando um ambiente de execução de memória RAM alta!")

INFO:root:Seu ambiente de execução tem  13.6 gigabytes de RAM disponível

INFO:root:Para habilitar um tempo de execução de RAM alta, selecione menu o ambiente de execução> "Alterar tipo de tempo de execução"
INFO:root:e selecione High-RAM. Então, execute novamente está célula


## 1.6 Monta uma pasta no google drive para carregar os arquivos de dados.

In [17]:
# import necessário
from google.colab import drive

# Monta o drive na pasta especificada
drive.mount("/content/drive")

Mounted at /content/drive


## 1.7 Instalação do spaCy

https://spacy.io/

Modelos do spaCy para português:
https://spacy.io/models/pt

In [18]:
# Instala o spacy
!pip install -U pip setuptools wheel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 5.3 MB/s 
     |████████████████████████████████| 1.2 MB 46.2 MB/s 
  Attempting uninstall: wheel
    Found existing installation: wheel 0.38.3
    Uninstalling wheel-0.38.3:
      Successfully uninstalled wheel-0.38.3
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.


In [19]:
# Instala uma versão específica
!pip install -U spacy==3.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.6/660.6 kB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 50.2 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.1.1
    Uninstalling typing_extensions-4.1.1:
      Successfully uninstalled typing_extensions-4.1.1
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.2
    Uninstalling pydantic-1.10.2:
      Successfully uninstalled pydantic-1.10.2
  Attempting uninstall: thinc
    Found existing installation: thinc 8.1.5
    Uninstalling thinc-8.1.5:
      Successfully uninstalled thinc-8.1.5
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.2
    Uninstalling spacy-3.4.2:
      Successfully uninstalled spacy-3.4.2
ERROR

## 1.8 Instalação do Gensim

Instalando o gensim no Google Colaboratory.

No Jupiter Notebook executar através "Anaconda Prompt".


In [20]:
#!pip install -U gensim
!pip install -U gensim==4.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 38.2 MB/s eta 0:00:00
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


# 2 Parametrização

## Gerais

## Específicos

Parâmetros do modelo

In [21]:
# Definição dos parâmetros do Modelo.
model_args = ModeloArgumentosMedida(     
    modelo_spacy = "en_core_web_lg",
    #modelo_spacy = "en_core_web_md",
    #modelo_spacy = "en_core_web_sm",

    versao_modelo_spacy = "3.2.0",
    
    sentenciar_documento = True,
    do_lower_case = False, # default True  
)

## Nome do diretório dos arquivos de dados

In [22]:
# Diretório do cohebert
DIRETORIO_COHEBERT = "COHQUAD_IN_EN"

## Define o caminho para os arquivos de dados

In [23]:
# Diretório local para os arquivos pré-processados
DIRETORIO_LOCAL = "/content/" + DIRETORIO_COHEBERT + "/"

# Diretório no google drive com os arquivos pré-processados
DIRETORIO_DRIVE = "/content/drive/MyDrive/Colab Notebooks/Data/" + DIRETORIO_COHEBERT + "/"

# 3 spaCy

## 3.1 Download arquivo modelo

https://spacy.io/models/pt

### Função download modelo spaCy

In [24]:
def downloadSpacy(model_args):
    """
      Realiza o download do arquivo do modelo para o diretório corrente.
    
      Parâmetros:
        `model_args` - Objeto com os argumentos do modelo.       
    """
    # Verifica se existe o diretório base
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()
        
    # Nome arquivo spacy
    ARQUIVO_MODELO_SPACY = model_args.modelo_spacy
    # Versão spaCy
    VERSAO_SPACY = "-" + model_args.versao_modelo_spacy
    # Nome arquivo compactado
    NOME_ARQUIVO_MODELO_COMPACTADO = ARQUIVO_MODELO_SPACY + VERSAO_SPACY + ".tar.gz"
    
    # Url do arquivo
    URL_ARQUIVO_MODELO_COMPACTADO = "https://github.com/explosion/spacy-models/releases/download/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY + "/" + NOME_ARQUIVO_MODELO_COMPACTADO

    # Realiza o download do arquivo do modelo
    logging.info("Download do arquivo do modelo do spaCy.")
    downloadArquivo(URL_ARQUIVO_MODELO_COMPACTADO, DIRETORIO_COHEBERT + "/" + NOME_ARQUIVO_MODELO_COMPACTADO)

## 3.2 Descompacta o arquivo do modelo

### Função descompacta modelo spaCy

In [25]:
# Import das bibliotecas.
import tarfile # Biblioteca de descompactação

def descompactaSpacy(model_args):
    """
      Descompacta o arquivo do modelo.
    
      Parâmetros:
        `model_args` - Objeto com os argumentos do modelo.       
    """
    
    # Verifica se existe o diretório base do cohebert e retorna o nome do diretório
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()
    
    # Nome arquivo spacy
    ARQUIVO_MODELO_SPACY = model_args.modelo_spacy
    # Versão spaCy
    VERSAO_SPACY = "-" + model_args.versao_modelo_spacy
    
    # Nome do arquivo a ser descompactado
    NOME_ARQUIVO_MODELO_COMPACTADO = DIRETORIO_COHEBERT + "/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY + ".tar.gz"
    
    logging.info("Descompactando o arquivo do modelo do spaCy.")
    arquivo_tar = tarfile.open(NOME_ARQUIVO_MODELO_COMPACTADO, "r:gz")    
    arquivo_tar.extractall(DIRETORIO_COHEBERT)    
    arquivo_tar.close()
    
    # Apaga o arquivo compactado
    if os.path.isfile(NOME_ARQUIVO_MODELO_COMPACTADO):        
        os.remove(NOME_ARQUIVO_MODELO_COMPACTADO)

## 3.3 Carrega o modelo

### Função carrega modelo spaCy

In [26]:
# Import das bibliotecas.
import spacy # Biblioteca do spaCy

def carregaSpacy(model_args):
    """
    Realiza o carregamento do Spacy.
    
    Parâmetros:
      `model_args` - Objeto com os argumentos do modelo.           
    """
    
    # Verifica se existe o diretório base
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()
                  
    # Nome arquivo spacy
    ARQUIVO_MODELO_SPACY = model_args.modelo_spacy
    # Versão spaCy
    VERSAO_SPACY = "-" + model_args.versao_modelo_spacy
    # Caminho raoz do modelo do spaCy
    DIRETORIO_MODELO_SPACY =  DIRETORIO_COHEBERT + "/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY

    # Verifica se o diretório existe
    if os.path.exists(DIRETORIO_MODELO_SPACY) == False:
        # Realiza o download do arquivo modelo do spaCy
        downloadSpacy(model_args)
        # Descompacta o spaCy
        descompactaSpacy(model_args)

    # Diretório completo do spaCy
    DIRETORIO_MODELO_SPACY = DIRETORIO_COHEBERT + "/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY + "/" + ARQUIVO_MODELO_SPACY + "/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY + "/"

    # Carrega o spaCy. Necessário somente "tagger" para encontrar os substantivos
    nlp = spacy.load(DIRETORIO_MODELO_SPACY)
    logging.info("spaCy carregado.")

    # Retorna o spacy carregado
    return nlp 

### Carrega o modelo spaCy


In [27]:
# Carrega o modelo spaCy
nlp = carregaSpacy(model_args)

INFO:root:Diretório Cohebert criado: COHQUAD_IN_EN
INFO:root:Download do arquivo do modelo do spaCy.
INFO:root:Download do arquivo: COHQUAD_IN_EN/en_core_web_lg-3.2.0.tar.gz.


  0%|          | 0.00/777M [00:00<?, ?B/s]

INFO:root:Descompactando o arquivo do modelo do spaCy.
INFO:root:spaCy carregado.


## 3.4 Funções auxiliares spaCy

### getStopwords

Recupera as stopwords do spaCy

In [28]:
def getStopwords(nlp):
    """
      Recupera as stop words do nlp(Spacy).
    
      Parâmetros:
        `nlp` - Um modelo spaCy carregado.           
    """
    
    spacy_stopwords = nlp.Defaults.stop_words

    return spacy_stopwords 

Lista dos stopwords

In [29]:
logging.info("Quantidade de stopwords: {}.".format(len(getStopwords(nlp))))

print(getStopwords(nlp))

INFO:root:Quantidade de stopwords: 326.


{'nevertheless', 'part', 'other', 'few', 'former', 'herein', 'used', '‘s', 'indeed', 'whether', 'therefore', 'such', 'thereupon', 'us', 'really', 'as', 'of', 'am', 'and', '‘re', 'whereby', 'sometimes', 'next', 'eleven', 'everyone', 'though', '‘ll', 'all', 'amount', '’m', 'anything', '’ve', 'because', 'my', 'becomes', "n't", 'since', 'take', 'whenever', 'elsewhere', 'upon', 'down', 'but', 'otherwise', 'doing', 'somewhere', 'back', 'noone', 'thereby', 'bottom', 'further', 'then', 'ca', 'should', 'nor', 'same', 'go', 'ever', 'thence', 'everything', 'does', 'becoming', 'into', 'every', 'everywhere', "'ve", 'never', 'via', 'what', "'m", 'your', 'here', 're', '‘ve', 'enough', 'too', 'well', 'where', 'please', 'thru', 'above', 'seem', 'somehow', 'twenty', 'on', 'i', 'formerly', 'at', 'for', 'onto', 'just', 'anywhere', 'with', 'thus', 'empty', 'them', '’re', 'might', 'wherever', 'several', 'something', 'without', 'many', 'often', 'even', 'four', 'whom', 'very', "'re", 'under', 'in', 'the', 'sh

### getVerbos
Localiza os verbos da sentença

In [30]:
# Import das bibliotecas.
import spacy   
from spacy.util import filter_spans
from spacy.matcher import Matcher

# (verbo normal como auxilar ou auxilar) + vários verbos auxiliares +verbo principal ou verbo auxiliar
gramaticav1 =  [
                {"POS": "AUX", "OP": "?", "DEP": {"IN": ["aux","aux:pass"]}},  #verbo auxiliar                                  
                {"POS": "VERB", "OP": "?", "DEP": {"IN": ["ROOT","aux","xcomp","aux:pass"]}},  #verbo normal como auxiliar
                {"POS": "AUX", "OP": "*", "DEP": {"IN": ["aux","xcomp","aux:pass"]}},  #verbo auxiliar   
                {"POS": "VERB", "OP": "+"}, #verbo principal
                {"POS": "AUX", "OP": "?", "DEP": {"IN": ["cop","aux","xcomp","aux:pass"]}},  #verbo auxiliar
               ] 

# verbo auxiliar + verbo normal como auxiliar + conjunção com preposição + verbo
gramaticav2 =  [               
                {"POS": "AUX", "OP": "?", "DEP": {"IN": ["aux","aux:pass"]}},  #verbo auxiliar                   
                {"POS": "VERB", "OP": "+", "DEP": {"IN": ["ROOT"]}},  #verbo principal       
                {"POS": "SCONJ", "OP": "+", "DEP": {"IN": ["mark"]}}, #conjunção com preposição
                {"POS": "VERB", "OP": "+", "DEP": {"IN": ["xcomp"]}}, #verbo normal como complementar
               ] 

#Somente verbos auxiliares
gramaticav3 =  [
                {"POS": "AUX", "OP": "?"},  #Verbos auxiliar 
                {"POS": "AUX", "OP": "?", "DEP": {"IN": ["cop"]}},  #Verbos auxiliar de ligação (AUX+(cop))
                {"POS": "ADJ", "OP": "+", "DEP": {"IN": ["ROOT"]}}, 
                {"POS": "AUX", "OP": "?"}  #Verbos auxiliar 
               ] 

matcherv = Matcher(nlp.vocab)
         
matcherv.add("frase verbal", [gramaticav1])
matcherv.add("frase verbal", [gramaticav2])
matcherv.add("frase verbal", [gramaticav3])

#Retorna a Frase Verbal
def getVerbos(periodo):    
  #Processa o período
  doc1 = nlp(periodo.text)
  
  # Chama o mather para encontrar o padrão
  matches = matcherv(doc1)

  padrao = [doc1[start:end] for _, start, end in matches]

  #elimina as repetições e sobreposições
  #return filter_spans(padrao)
  lista1 = filter_spans(padrao)

  # Converte os itens em string
  lista2 = []
  for x in lista1:
      lista2.append(str(x))
  
  return lista2

### getDicPOSQtde

Conta as POS Tagging de uma sentença

In [31]:
def getDicPOSQtde(sentenca):

    # Verifica se o sentenca não foi processado pelo spaCy  
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Retorna inteiros que mapeiam para classes gramaticais
  conta_dicionarios = doc.count_by(spacy.attrs.IDS["POS"])

  # Dicionário com as tags e quantidades
  novodic = dict()
  
  for pos, qtde in conta_dicionarios.items():
    classe_gramatical = doc.vocab[pos].text
    novodic[classe_gramatical] = qtde

  return novodic

In [32]:
def getDicTodasPOSQtde(sentenca):

    # Verifica se o sentenca não foi processado pelo spaCy  
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Retorna inteiros que mapeiam para classes gramaticais
  conta_dicionarios = doc.count_by(spacy.attrs.IDS["POS"])

  # Dicionário com as tags e quantidades    
  novodic = {"PRON":0, "VERB":0, "PUNCT":0, "DET":0, "NOUN":0, "AUX":0, "CCONJ":0, "ADP":0, "PROPN":0, "ADJ":0, "ADV":0, "NUM":0, "SCONJ":0, "SYM":0, "SPACE":0, "INTJ":0, "X": 0}
    
  for pos, qtde in conta_dicionarios.items():
    classe_gramatical = doc.vocab[pos].text
    novodic[classe_gramatical] = qtde

  return novodic

### getDicTodasPOSQtde

Conta as POS Tagging de uma sentença

In [33]:
def getDicTodasPOSQtde(lista):

  # Dicionário com as tags e quantidades
  conjunto = {"PRON":0, "VERB":0, "PUNCT":0, "DET":0, "NOUN":0, "AUX":0, "CCONJ":0, "ADP":0, "PROPN":0, "ADJ":0, "ADV":0, "NUM":0, "SCONJ":0, "SYM":0, "SPACE":0, "INTJ": 0}

  for x in lista:
    valor = conjunto.get(x)
    if valor != None:
      conjunto[x] = valor + 1
    else:
      conjunto[x] = 1

  return conjunto

### getSomaDic

Soma os valores de dicionários com as mesmas chaves.

In [34]:
from collections import Counter
from functools import reduce

def atualizaValor(a,b):
    a.update(b)
    return a

def getSomaDic(lista):
    
  # Soma os dicionários da lista
  novodic = reduce(atualizaValor, (Counter(dict(x)) for x in lista))
 
  return novodic

### getTokensSentenca

Retorna a lista de tokens da sentenca.

In [35]:
def getTokensSentenca(sentenca):

    # Verifica se o sentenca não foi processado pelo spaCy  
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Lista dos tokens
  lista = []

  # Percorre a sentença adicionando os tokens
  for token in doc:    
    lista.append(token.text)

  return lista

### getPOSTokensSentenca

Retorna a lista das POS-Tagging dos tokens da sentenca.

In [36]:
def getPOSTokensSentenca(sentenca):

  # Verifica se o sentenca não foi processado pelo spaCy  
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Lista dos tokens
  lista = []

  # Percorre a sentença adicionando os tokens
  for token in doc:    
    lista.append(token.pos_)

  return lista

### getListaTokensPOSSentenca

Retorna duas listas uma com os tokens e a outra com a POS-Tagging dos tokens da sentenca.

In [37]:
def getListaTokensPOSSentenca(sentenca):
  # Verifica se o sentenca não foi processado pelo spaCy  
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Lista dos tokens
  listatokens = []
  listapos = []

  # Percorre a sentença adicionando os tokens e as POS
  for token in doc:    
    listatokens.append(token.text)
    listapos.append(token.pos_)
    
  return listatokens, listapos

### Tradução das tags

Tags de palavras universal

https://universaldependencies.org/u/pos/

Detalhes das tags em português:
http://www.dbd.puc-rio.br/pergamum/tesesabertas/1412298_2016_completo.pdf

In [38]:
#dicionário que contêm pos tag universal e suas explicações
palavra_universal_dict = {
  "X"    : "Outro",
  "VERB" : "Verbo ",
  "SYM"  : "Símbolo",
  "CONJ" : "Conjunção",
  "SCONJ": "Conjunção subordinativa",
  "PUNCT": "Pontuação",
  "PROPN": "Nome próprio",
  "PRON" : "Pronome substativo",
  "PART" : "Partícula, morfemas livres",
  "NUM"  : "Numeral",
  "NOUN" : "Substantivo",
  "INTJ" : "Interjeição",
  "DET"  : "Determinante, Artigo e pronomes adjetivos",
  "CCONJ": "Conjunção coordenativa",
  "AUX"  : "Verbo auxiliar",
  "ADV"  : "Advérbio",
  "ADP"  : "Preposição",
  "ADJ"  : "Adjetivo"
}
  
#Explica a POS
def getPOSPalavraUniversalTraduzido(palavra):
  if palavra in palavra_universal_dict.keys():
      traduzido = palavra_universal_dict[palavra]
  else:
      traduzido = "NA" 
  return traduzido

### getSentencaSemStopWord

Retorna uma lista dos tokens sem as stopwords.

In [39]:
def getSentencaSemStopWord(sentenca, stopwords):

  # Lista dos tokens
  lista = []

  # Percorre os tokens da sentença
  for i, token in enumerate(sentenca):

    # Verifica se o token é uma stopword
    if token.lower() not in stopwords:
      lista.append(token)

  # Retorna o documento
  return lista

### getSentencaSalientePOS

Retorna uma lista das palavras do tipo especificado.

In [40]:
def getSentencaSalientePOS(sentenca, pos, classe_saliente=["NOUN"]):
  
  # Lista dos tokens
  lista = []

  # Percorre a sentença
  for i, token in enumerate(sentenca):

    # Verifica se o token é do tipo especificado
    if pos[i] in classe_saliente:
      lista.append(token)

  # Retorna o documento
  return lista

###removeStopWords

Remove as stopwords de um documento ou senteça.

In [41]:
def removeStopWord(documento, stopwords):
  
  # Remoção das stopwords do documento
  documentoSemStopwords = [palavra for palavra in documento.split() if palavra.lower() not in stopwords]

  # Concatena o documento sem os stopwords
  documento_limpo = " ".join(documentoSemStopwords)

  # Retorna o documento
  return documento_limpo

### getTokensSemStopword

Retira as stopswords de lista de tokens

In [42]:
def getTokensSemStopword(tokens, spacy_stopwords=getStopwords(nlp)):
    """
      Retira os tokens da lista de tokens tokens que estão na lista de stopword.
      A lista de tokens pode ou não estar dentro de uma outra lista.
    
      Parâmetros:
        `tokens` - Uma lista com os tokens ou uma lista de lista de tokens.
        `spacy_stopwords` - Uma lista com as stopword. 
    """
    
    # Verifica se é uma lista de palavras(str) ou ou uma lista de lista
    if type(tokens[0]) is str:
      lista_tokens = [tokens]
    else:
      lista_tokens = tokens
      
    # Lista de retorno
    lista_tokens_sem_stopwords = []  

    # Percorre a lista de tokens
    for texto in lista_tokens:

      # Lista dos tokens sem as stopwords
      tokens_sem_stopwords = []
      
      # Percorre os tokens    
      for token in texto:
        # Verifica se o toke não está na lista de stopwords para adicionar a nova lista
        if token not in spacy_stopwords:
          tokens_sem_stopwords.append(token)
      
      # Adiciona a lista de tokens sem stopwords na lista de retorno se tiver uma palavra
      if len(tokens_sem_stopwords) != 0:
        lista_tokens_sem_stopwords.append(tokens_sem_stopwords)

    if type(tokens[0]) is str:      
      return lista_tokens_sem_stopwords[0]
    else:
      return lista_tokens_sem_stopwords

### getSentencasTexto

Retorna a lista de tokens de uma lista de textos.

In [43]:
def getSentencasTexto(textos, nlp = nlp):

  """
     Sentencia um texto ou uma lista de textos.
    
     Parâmetros:
      `textos` - Um texto(str) ou uma lista de textos.
      `nlp` - Modelo spacy carregado.

  """

  # Verifica se é um texto é str ou uma lista de texto
  if type(textos) is str:
    lista_texto = [textos]
  else:
    lista_texto = textos

  # Lista dos tokens
  lista_sentencas = []

  for texto in lista_texto:

    # Sentencia o documento
    doc = nlp(texto)
      
    # Percorre as sentenças do documento
    for sentenca in doc.sents:   

        lista_sentencas.append(str(sentenca))
      
  # Verifica o tipo documento para o tipo de retorno
  if type(textos) is str:
    return lista_sentencas[0]
  else:
    return lista_sentencas

### getSentencasMinusculo

Retorna a lista das sentencas do texto em minúsculo.

In [44]:
def getSentencasMinusculo(textos):

  """
     Sentencia um texto ou uma lista de textos em minusculo.
    
     Parâmetros:
      `textos` - Um texto(str) ou uma lista de textos.

  """

  # Verifica se é um texto é str ou uma lista de texto
  if type(textos) is str:
    lista_texto = [textos]
  else:
    lista_texto = textos

  # Lista dos tokens
  lista_sentencas = []

  for texto in lista_texto:

    lista_sentencas.append(str(texto).lower())
      
  # Verifica o tipo documento para o tipo de retorno
  if type(textos) is str:
    return lista_sentencas[0]
  else:
    return lista_sentencas

### getTokensTexto

Retorna a lista de tokens do texto.

In [45]:
def getTokensTexto(textos, nlp = nlp):

  """
     Tokeniza um texto ou uma lista de textos.
    
     Parâmetros:
      `textos` - Um texto(str) ou uma lista de textos.
  """

  # Verifica se é um texto é str ou uma lista de texto
  if type(textos) is str:
    lista_texto = [textos]
  else:
    lista_texto = textos

  # Lista de retorno
  lista_tokens_texto = []

  # Percorre a lista de texto
  for texto in lista_texto:

    # Verifica se o sentenca não foi processado pelo spaCy  
    if type(texto) is not spacy.tokens.doc.Doc:
        # Realiza o parsing no spacy
        doc = nlp(texto)
    else:
        doc = texto

    # Lista dos tokens
    lista_tokens = []

    # Percorre a sentença adicionando os tokens
    for token in doc:    
      lista_tokens.append(token.text)
    
    # Adiciona a lista de tokens na lista de sentenças
    lista_tokens_texto.append(lista_tokens)

  # Verifica o tipo documento para o tipo de retorno
  if type(textos) is str:
    return lista_tokens_texto[0]
  else:
    return lista_tokens_texto

### removerPontuacao

Remove pontuação

In [46]:
def removerPontuacao(textos):
    
    """https://spacy.io/api/annotation"""

    textos_saida = []

    for texto in textos:
        
        doc = nlp(" ".join(texto)) 

        sentenca = []
        for token in doc:
          if token.pos_ not in ['PUNCT']:
              sentenca.append(token.text)

        if len(sentenca) != 0:
          textos_saida.append(sentenca)

    return textos_saida

### relevantes

Palavras relevantes

In [47]:
def relevantes(textos, postags_permitidas=['VER', 'AUX', 'NOUN']):
    
    """https://spacy.io/api/annotation"""

    textos_saida = []

    for texto in textos:
        
        doc = nlp(" ".join(texto)) 
      
        sentenca = []
        for token in doc:
          if token.pos_ in postags_permitidas:
              sentenca.append(token.text)

        if len(sentenca) != 0:
          textos_saida.append(sentenca)

    return textos_saida

### lematizacao

Lematização do texto

In [48]:
def lematizacao(textos, postags_permitidas=['NOUN', 'ADJ', 'VERB', 'ADV']):
    
    """https://spacy.io/api/annotation"""

    textos_saida = []

    for texto in textos:
        doc = nlp(" ".join(texto)) 

        sentenca = []
        for token in doc:
          if token.pos_ in postags_permitidas:
              sentenca.append(token.lemma_)

        if len(sentenca) != 0:
          textos_saida.append(sentenca)

    return textos_saida

### preparaCorpus

In [49]:
# Import das biblitecas
import pandas as pd
import re
import gensim

def preparaCorpus(textos,                                     
                  sentenciaTexto=False,
                  tornaMinusculo=False,
                  removePontuacao=False, 
                  removeStopwords=False, 
                  bigramas=False, 
                  trigramas=False,
                  somenteRelevante=False,
                  postag_relevante=['VERB', 'AUX', 'NOUN'],
                  lematizar=False,                  
                  postag_lema=['NOUN', 'ADJ', 'VERB', 'ADV']):

    # Verifica se é um textos é str ou uma lista de texto
    if type(textos) is str:
      # Sentencia o texto
      lista_sentencas = [textos]
    else:
      lista_sentencas = textos
    
    # Converte o texto em uma lista de sentencas
    if sentenciaTexto==True:
      lista_sentencas = getSentencasTexto(lista_sentencas)

    # Converte o texto em minúsuclo
    if tornaMinusculo==True:
      lista_sentencas = getSentencasMinusculo(lista_sentencas)
    
    # tokeniza o texto
    lista_sentencas_palavras = getTokensTexto(lista_sentencas)

    # Remove a pontuação 
    if removePontuacao==True:
        lista_sentencas_palavras = removerPontuacao(lista_sentencas_palavras)        

    # Remove as stop words
    if removeStopwords==True:
      lista_sentencas_palavras = getTokensSemStopword(lista_sentencas_palavras)

    # Criar bigramas ou trigramas
    if bigramas==True:
      # Construa os modelos de bigramas
      bigram = gensim.models.Phrases(lista_sentencas_palavras, min_count=5, threshold=100) # max_topicse mais alto menos frases.
      # Maneira mais rápida de obter uma frase batida como um trigrama/bigrama
      bigram_mod = gensim.models.phrases.Phraser(bigram)
      lista_sentencas_palavras = [bigram_mod[doc] for doc in lista_sentencas_palavras]
    
    if trigramas==True:      
      # Construa os modelos de bigramas
      bigram = gensim.models.Phrases(lista_sentencas_palavras, min_count=5, threshold=100) # max_topicse mais alto menos frases.
      # Maneira mais rápida de obter uma frase batida como um trigrama/bigrama
      bigram_mod = gensim.models.phrases.Phraser(bigram)
      # Construa os modelos de trigramas
      trigram = gensim.models.Phrases(bigram[lista_sentencas_palavras], threshold=100)
      # Maneira mais rápida de obter uma frase batida como um trigrama/bigrama    
      trigram_mod = gensim.models.phrases.Phraser(trigram)   
      lista_sentencas_palavras = [trigram_mod[bigram_mod[doc]] for doc in lista_sentencas_palavras]   
    
    # Somente palavras relevantes
    if somenteRelevante==True:      
      lista_sentencas_palavras = relevantes(lista_sentencas_palavras, postags_permitidas=postag_relevante)
    
    # Faça a lematização mantendo apenas para noun, adj, vb, adv
    if lematizar==True:      
      lista_sentencas_palavras = lematizacao(lista_sentencas_palavras, postags_permitidas=postag_lema)

    return lista_sentencas_palavras

# 4 Corpus Específico

## 4.1 Especifica os nomes dos arquivos do corpus

In [50]:
# Nome do arquivo
NOME_ARQUIVO_CORPUS = "corpus_especifico.csv"
NOME_ARQUIVO_CORPUS_COMPACTADO = "corpus_especifico.zip"

## 4.2 Cria o diretório local para receber os dados

In [51]:
# Biblioteca para acessar o sistema de arquivos
import os

#Cria o diretório para receber os arquivos Originais e Perturbados
# Diretório a ser criado
dirbase = DIRETORIO_LOCAL[:-1]

if not os.path.exists(dirbase):  
    # Cria o diretório
    os.makedirs(dirbase)    
    logging.info("Diretório criado: {}.".format(dirbase))
else:    
    logging.info("Diretório já existe: {}.".format(dirbase))

INFO:root:Diretório já existe: /content/COHQUAD_IN_EN.


## 4.3 Conteúdo do Arquivo do Corpus

In [52]:
documentos = [
# Pilhas https://en.wikipedia.org/wiki/Stack_(abstract_data_type)
['pilha','wikipedia','For the use of the term LIFO in accounting, see LIFO (accounting).'],
['pilha','wikipedia','For the use of the term pushdown in strength training, see Pushdown (exercise).'],
['pilha','wikipedia','For other uses, see Stack (disambiguation).'],
['pilha','wikipedia','Similar to a stack of plates, adding or removing is only possible at the top.'],
['pilha','wikipedia','Simple representation of a stack runtime with push and pop operations.'],
['pilha','wikipedia','In computer science, a stack is an abstract data type that serves as a collection of elements, with two main operations:'],
['pilha','wikipedia','Push, which adds an element to the collection, and Pop, which removes the most recently added element that was not yet removed.'],
['pilha','wikipedia','Additionally, a peek operation can, without modifying the stack, return the value of the last element added.'],
['pilha','wikipedia','Calling this structure a stack is by analogy to a set of physical items stacked one atop another, such as a stack of plates.'],
['pilha','wikipedia','The order in which an element added to or removed from a stack is described as last in, first out, referred to by the acronym LIFO.[nb 1] As with a stack of physical objects, this structure makes it easy to take an item off the top of the stack, but accessing a datum deeper in the stack may require taking off multiple other items first.'],
['pilha','wikipedia','Considered as a linear data structure, or more abstractly a sequential collection, the push and pop operations occur only at one end of the structure, referred to as the top of the stack.'],
['pilha','wikipedia','This data structure makes it possible to implement a stack as a singly linked list and as a pointer to the top element.'],
['pilha','wikipedia','A stack may be implemented to have a bounded capacity.'],
['pilha','wikipedia','If the stack is full and does not contain enough space to accept another element, the stack is in a state of stack overflow.'],
['pilha','wikipedia','A stack is needed to implement depth-first search.'],

# Pilha Thomas Cormen
['pilha','Thomas Cormen','Stacks and queues are dynamic sets in which the element removed from the set by the DELETE operation is prespecified.'],
['pilha','Thomas Cormen','In a stack, the element deleted from the set is the one most recently inserted: the stack implements a last-in, first-out, or LIFO, policy.'],
['pilha','Thomas Cormen','Similarly, in a queue, the element deleted is always the one that has been in the set for the longest time: the queue implements a first-in, first-out, or FIFO, policy.'],
['pilha','Thomas Cormen','There are several efficient ways to implement stacks and queues on a computer.'],
['pilha','Thomas Cormen','In this section we show how to use a simple array to implement each.'],
['pilha','Thomas Cormen','The INSERT operation on a stack is often called PUSH, and the DELETE operation, which does not take an element argument, is often called POP.'],
['pilha','Thomas Cormen','These names are allusions to physical stacks, such as the spring-loaded stacks of plates used in cafeterias.'],
['pilha','Thomas Cormen','The order in which plates are popped from the stack is the reverse of the order in which they were pushed onto the stack, since only the top plate is accessible.'],
['pilha','Thomas Cormen','As Figure 10.1 shows, we can implement a stack of at most n elements with an array S[1..n].'],
['pilha','Thomas Cormen','The array has an attribute S:top that indexes the most recently inserted element.'],
['pilha','Thomas Cormen','The stack consists of elements S[1..S.top], where S[1] is the element at the bottom of the stack and S[S.top] is the element at the top.'],
['pilha','Thomas Cormen','When S.top = 0, the stack contains no elements and is empty.'],
['pilha','Thomas Cormen','We can test to see whether the stack is empty by query operation STACK-EMPTY.'],
['pilha','Thomas Cormen','If we attempt to pop an empty stack, we say the stack underflows, which is normally an error.'],
['pilha','Thomas Cormen','If S.top exceeds n, the stack overflows.'],
['pilha','Thomas Cormen','(In our pseudocode implementation, we don’t worry about stack overflow.)'],
['pilha','Thomas Cormen','We can implement each of the stack operations with just a few lines of code:'],
['pilha','Thomas Cormen','STACK-EMPTY(S)'],
['pilha','Thomas Cormen','1 if S.top == 0'],
['pilha','Thomas Cormen','2 return TRUE'],
['pilha','Thomas Cormen','3 else return FALSE'],
['pilha','Thomas Cormen','PUSH(S,x)'],
['pilha','Thomas Cormen','1 S.top= S.top + 1'],
['pilha','Thomas Cormen','2 S[S.top] = x'],
['pilha','Thomas Cormen','POP(S)'],
['pilha','Thomas Cormen','1 if STACK-EMPTY(S)'],
['pilha','Thomas Cormen','2 error \“underflow\”'],
['pilha','Thomas Cormen','3 else S.top = S.top -1'],
['pilha','Thomas Cormen','4 return S[S.top + 1]'],
['pilha','Thomas Cormen','Figure 10.1 shows the effects of the modifying operations PUSH and POP.'],
['pilha','Thomas Cormen','Each of the three stack operations takes O(1) time.'],
['pilha','Thomas Cormen','Figure 10.2 A queue implemented using an array Q[1..12] Queue elements appear only in the lightly shaded positions.'],
['pilha','Thomas Cormen','(a) The queue has 5 elements, in locations Q[7..11].'],
['pilha','Thomas Cormen','(b) The configuration of the queue after the calls ENQUEUE(Q,17), ENQUEUE(Q,3), and ENQUEUE(Q,5).'],
['pilha','Thomas Cormen','(c) The configuration of the queue after the call DEQUEUE(Q) returns the key value 15 formerly at the head of the queue.'],
['pilha','Thomas Cormen','The new head has key 6.'],


# Fila https://en.wikipedia.org/wiki/Queue_(abstract_data_type)
['pilha','wikipedia','In computer science, a queue is a collection of entities that are maintained in a sequence and can be modified by the addition of entities at one end of the sequence and the removal of entities from the other end of the sequence.'],
['pilha','wikipedia','By convention, the end of the sequence at which elements are added is called the back, tail, or rear of the queue, and the end at which elements are removed is called the head or front of the queue, analogously to the words used when people line up to wait for goods or services.'],
['pilha','wikipedia','The operation of adding an element to the rear of the queue is known as enqueue, and the operation of removing an element from the front is known as dequeue.'],
['pilha','wikipedia','Other operations may also be allowed, often including a peek or front operation that returns the value of the next element to be dequeued without dequeuing it.'],
['pilha','wikipedia','The operations of a queue make it a first-in-first-out (FIFO) data structure.'],
['pilha','wikipedia','In a FIFO data structure, the first element added to the queue will be the first one to be removed.'],
['pilha','wikipedia','This is equivalent to the requirement that once a new element is added, all elements that were added before have to be removed before the new element can be removed.'],
['pilha','wikipedia','A queue is an example of a linear data structure, or more abstractly a sequential collection.'],
['pilha','wikipedia','Queues are common in computer programs, where they are implemented as data structures coupled with access routines, as an abstract data structure or in object-oriented languages as classes.'],
['pilha','wikipedia','Common implementations are circular buffers and linked lists.'],
['pilha','wikipedia','Queues provide services in computer science, transport, and operations research where various entities such as data, objects, persons, or events are stored and held to be processed later.'],
['pilha','wikipedia','In these contexts, the queue performs the function of a buffer.'],
['pilha','wikipedia','Another usage of queues is in the implementation of breadth-first search.'],
['pilha','wikipedia','Theoretically, one characteristic of a queue is that it does not have a specific capacity.'],
['pilha','wikipedia','Regardless of how many elements are already contained, a new element can always be added.'],
['pilha','wikipedia','It can also be empty, at which point removing an element will be impossible until a new element has been added again.'],
['pilha','wikipedia','Fixed-length arrays are limited in capacity, but it is not true that items need to be copied towards the head of the queue.'],
['pilha','wikipedia','The simple trick of turning the array into a closed circle and letting the head and tail drift around endlessly in that circle makes it unnecessary to ever move items stored in the array.'],
['pilha','wikipedia','If n is the size of the array, then computing indices modulo n will turn the array into a circle.'],
['pilha','wikipedia','This is still the conceptually simplest way to construct a queue in a high-level language, but it does admittedly slow things down a little, because the array indices must be compared to zero and the array size, which is comparable to the time taken to check whether an array index is out of bounds, which some languages do, but this will certainly be the method of choice for a quick and dirty implementation, or for any high-level language that does not have pointer syntax.'],
['pilha','wikipedia','The array size must be declared ahead of time, but some implementations simply double the declared array size when overflow occurs.'],
['pilha','wikipedia','Most modern languages with objects or pointers can implement or come with libraries for dynamic lists.'],
['pilha','wikipedia','Such data structures may have not specified a fixed capacity limit besides memory constraints.'],
['pilha','wikipedia','Queue overflow results from trying to add an element onto a full queue and queue underflow happens when trying to remove an element from an empty queue.'],
['pilha','wikipedia','A bounded queue is a queue limited to a fixed number of items.'],

# Pilha Thomas Cormen
['fila','Thomas Cormen','We call the INSERT operation on a queue ENQUEUE, and we call the DELETE operation DEQUEUE; like the stack operation POP, DEQUEUE takes no element argument.'],
['fila','Thomas Cormen','The FIFO property of a queue causes it to operate like a line of customers waiting to pay a cashier.'],
['fila','Thomas Cormen','The queue has a head and a tail.'],
['fila','Thomas Cormen','When an element is enqueued, it takes its place at the tail of the queue, just as a newly arriving customer takes a place at the end of the line.'],
['fila','Thomas Cormen','The element dequeued is always the one at the head of the queue, like the customer at the head of the line who has waited the longest.'],
['fila','Thomas Cormen','Figure 10.2 shows one way to implement a queue of at most n-1 elements using an array Q[1..n].'],
['fila','Thomas Cormen','The queue has an attribute Q:head that indexes, or points to, its head.'],
['fila','Thomas Cormen','The attribute Q:tail indexes the next location at which a newly arriving element will be inserted into the queue.'],
['fila','Thomas Cormen','The elements in the queue reside in locations Q.head;Q.head + 1;...;Q.tail - 1, where we “wrap around” in the sense that location 1 immediately follows location n in a circular order.'],
['fila','Thomas Cormen','When Q:head D Q:tail, the queue is empty.'],
['fila','Thomas Cormen','Initially, we have Q:head D Q:tail D 1.'],
['fila','Thomas Cormen','If we attempt to dequeue an element from an empty queue, the queue underflows.'],
['fila','Thomas Cormen','When Q.head = Q.tail + 1, the queue is full, and if we attempt to enqueue an element, then the queue overflows.'],
['fila','Thomas Cormen','In our procedures ENQUEUE and DEQUEUE, we have omitted the error checking for underflow and overflow.'],
['fila','Thomas Cormen','(Exercise 10.1-4 asks you to supply code that checks for these two error conditions.)'],
['fila','Thomas Cormen','The pseudocode assumes that n D Q:length.'],
['fila','Thomas Cormen','ENQUEUE(Q,x)'],
['fila','Thomas Cormen','1 Q[Q.tail] = x'],
['fila','Thomas Cormen','2 if Q.tail'],
['fila','Thomas Cormen','== Q.length'],
['fila','Thomas Cormen','3 Q.tail = 1'],
['fila','Thomas Cormen','4 else Q.tail = Q.tail + 1'],
['fila','Thomas Cormen','DEQUEUE(Q)'],
['fila','Thomas Cormen','1 x D Q[Q.head]'],
['fila','Thomas Cormen','2 if Q.head == Q.length'],
['fila','Thomas Cormen','3 Q.head = 1'],
['fila','Thomas Cormen','4 else Q.head D Q.head + 1'],
['fila','Thomas Cormen','5 return x'],
['fila','Thomas Cormen','Figure 10.2 shows the effects of the ENQUEUE and DEQUEUE operations.'],
['fila','Thomas Cormen','Each operation takes O.1/ time.']
]

print("Quantidade de documentos:", len(documentos))

Quantidade de documentos: 106


## 4.5 Cria o arquivo do corpus

In [53]:
# Import das bibliotecas.
import pandas as pd

# Cria o dataframe da lista
df_lista_sentencas = pd.DataFrame(documentos, columns = ["topico","fonte","sentenca"])
 
df_lista_sentencas.to_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_CORPUS, sep=";", index=False)

## 4.6 Compacta e copia o arquivo perturbado para uma pasta do GoogleDrive

Compacta o arquivo gerado da comparação para facilitar o envio para o GoogleDrive

In [54]:
!zip -o -q -j "$DIRETORIO_LOCAL$NOME_ARQUIVO_CORPUS_COMPACTADO" "$DIRETORIO_LOCAL$NOME_ARQUIVO_CORPUS"

logging.info("Terminei compactação.")

INFO:root:Terminei compactação.


Copia o arquivo compactado e os arquivos individuais para o GoogleDrive

In [55]:
# Import das bibliotecas.
import os
import datetime

# Se estiver executando no Google Colaboratory
if IN_COLAB:
     
    # Copia o arquivo perturbado
    !cp "$DIRETORIO_LOCAL$NOME_ARQUIVO_CORPUS_COMPACTADO" "$DIRETORIO_DRIVE"
    
    logging.info("Terminei a cópia.")

INFO:root:Terminei a cópia.


## 4.7 Carrega os dados

In [56]:
# Import das bibliotecas.
import pandas as pd

# Abre o arquivo e retorna o DataFrame
df_corpus = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_CORPUS, sep=";", encoding="UTF-8")

print(len(df_corpus))

106


In [57]:
df_corpus.sample(5)

,topico,fonte,sentenca
1,pilha,wikipedia,For the use of the term pushdown in strength t...
31,pilha,Thomas Cormen,We can implement each of the stack operations ...
52,pilha,wikipedia,"By convention, the end of the sequence at whic..."
105,fila,Thomas Cormen,Each operation takes O.1/ time.
32,pilha,Thomas Cormen,STACK-EMPTY(S)


# 5 Finalização

## 5.1 Tempo final de processamento



In [58]:
# Pega o tempo atual menos o tempo do início do processamento.
final_processamento = time.time()
tempo_total_processamento = formataTempo(final_processamento - inicio_processamento)

print("")
print("  Tempo processamento:  {:} (h:mm:ss)".format(tempo_total_processamento))


  Tempo processamento:  0:02:21 (h:mm:ss)
